## Encontrando a Localizacao de Empresas Desenvolvedoras de App na Play Store

In [2]:
import requests
import bs4
import time
import folium

In [3]:
main_url = 'https://play.google.com'

In [2]:
# PAGINA PRINCIPAL

url = 'https://play.google.com/store/apps/category/GAME_ROLE_PLAYING?hl=pt_BR'
req = requests.get(url)

htmlPage = bs4.BeautifulSoup(req.content, features='html.parser')

apps_subcategories = htmlPage.find('div',{'class':'N4FjMb'})
links = apps_subcategories.find_all('a')
for link in links:
    if link.text == 'Veja mais':
        print(link.get('href'))

In [7]:
# CATEGORIA SELECIONADA 
url = 'https://play.google.com/store/search?q=futebol&c=apps&hl=pt_BR'
req = requests.get(url)
htmlPage = bs4.BeautifulSoup(req.content, features='html.parser')

links_app = [main_url + app.get('href') for app in htmlPage.find_all('a',{'class':'poRVub'})]

In [8]:
# PEGA OS DADOS DOS APLICATIVOS
applications_data = []
total = 0
for app in links_app:
    try:
        req = requests.get(app)
        if req.status_code == 200:
            htmlPage = bs4.BeautifulSoup(req.content, features='html.parser')
            time.sleep(3)

            app_info = {'name': '', 'dev_location': '','website': '', 'email': '', 'privacy_policy': '', 'rating': '', 'developer_company': ''}

            # nome do aplicativo
            name = htmlPage.find('h1').text
#             print('name saved')

            #pega os dados de desenvolvedor
            dev_area = htmlPage.find_all('div',{'class':'hAyfc'})[-1].find('span',{'class':'htlgb'})
            website, email, privacy_policy = [link.get('href') for link in dev_area.find_all('a')]
            dev_location = dev_area.find_all('div')[-1].text
#             print('dev saved')

            # pega a classificacao de idade
            rating = htmlPage.find_all('div',{'class':'hAyfc'})[5].find_all('span')[1].find('div').text
#             print('rating saved')

            # pega a empresa desenvolvedora 
            developer_company = htmlPage.find_all('div',{'class':'hAyfc'})[-2].find_all('span',{'class':'htlgb'})[-1].text
#             print('dev company saved')

            app_info['name'], app_info['dev_location'] ,app_info['website'], app_info['email'], app_info['privacy_policy'], app_info['rating'], app_info['developer_company'] = name, dev_location,website, email, privacy_policy, rating, developer_company 
            applications_data.append(app_info)

    except Exception as error:
        print(type(error))
        print(error)
    
    total += 1
    print('data from {} apps saved'.format(total))
    time.sleep(3)

data from 1 apps saved
data from 2 apps saved
data from 3 apps saved
data from 4 apps saved
data from 5 apps saved
data from 6 apps saved
data from 7 apps saved
data from 8 apps saved
data from 9 apps saved
data from 10 apps saved
data from 11 apps saved
data from 12 apps saved
data from 13 apps saved
data from 14 apps saved
data from 15 apps saved
data from 16 apps saved
data from 17 apps saved
data from 18 apps saved
data from 19 apps saved
data from 20 apps saved
data from 21 apps saved
data from 22 apps saved
data from 23 apps saved
data from 24 apps saved
data from 25 apps saved
data from 26 apps saved
data from 27 apps saved
data from 28 apps saved
data from 29 apps saved
data from 30 apps saved
data from 31 apps saved
<class 'ValueError'>
not enough values to unpack (expected 3, got 2)
data from 32 apps saved
<class 'ValueError'>
not enough values to unpack (expected 3, got 2)
data from 33 apps saved
data from 34 apps saved
data from 35 apps saved
data from 36 apps saved
data fr

In [9]:
# CORRIGE OS ENDERECOS QUE ESTAO SEPARADOS EM LINHAS
for address in range(len(applications_data)):
    new_address = ' '.join(applications_data[address]['dev_location'].split('\n'))
    applications_data[address]['dev_location'] = {}
    applications_data[address]['dev_location']['address'] = new_address

In [10]:
# ABRE UM NAVEGADOR EM HEADLESS PARA FAZER AS PESQUISAS
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions

#instance headless
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--window-size-1920x1080')

driver = webdriver.Chrome(options=options, executable_path="./webdriver/chromedriver")

In [11]:
# FAZ UM BACKUP DOS DADOS COLETADOS
applications_saved = list(applications_data)

In [14]:
for application in range(len(applications_data)):
    try:
        name = applications_data[application]['dev_location']['address']

        # TRANSFORMA OS NOMES DOS LUGARES EM UMA STRING VALIDA PARA O GOOGLE MAPS
        addres_search = 'http://maps.google.com/?q=' + name.replace(' ','+')
        driver.get(addres_search)

        # ESPERA O GOOGLE MAPS CARREGAR O MAPA E PEGA A URL ATUAL QUE ESTARA COM OS DADOS DE GEOLOCALIZACAO
        time.sleep(6)
        url_atual = driver.current_url.split('/') #separa os elementos da url

        #latitude e longitude estarao na posicao 7 da lista
        if url_atual[6].startswith('@'): 
            url_atual = url_atual[6].split(',')[:2] #separa latitude e longitude
            url_atual[0] = url_atual[0].replace('@','') #tira o @ do inicio da latitude
            lat, long = url_atual #desmembra a lista e salva latitude e longitude no dict das localizacoes
            applications_data[application]['dev_location']['lat'], applications_data[application]['dev_location']['long'] = lat, long
        else:
            #se nao encontrar ira remover da lista
            applications_data.pop(application)
    except:
        pass

In [18]:
# CRIA UM MAPA
dev_companys = folium.Map()

In [19]:
# ADICIONA AS LOCALIZACOES
for company in applications_data:
    try:
        lat, long = float(company['dev_location']['lat']), float(company['dev_location']['long'])
        folium.Marker(location=[lat, long], popup=company['name']).add_to(dev_companys)
    except KeyError:
        pass

In [20]:
dev_companys

In [21]:
# SALVA O MAPA HTML
dev_companys.save('apps-play-store.html')